# Trabajo Práctico 2

In [ ]:
import importlib
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import pandas as pd
import tp2
from tp2.perceptron import ThresholdUnit, NonLinearUnit
import tp2.aux as tp2Aux
import pickle

In [ ]:
importlib.reload(tp2.perceptron)
importlib.reload(tp2Aux)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0)

### 1. Implemente un perceptrón simple que aprenda la función lógica AND de 2 y de 4 entradas. Lo mismo para la función lógica OR. Para el caso de 2 dimensiones, grafique la recta discriminadora y todos los vectores de entrada de la red.

#### AND de dos entradas

Tabla de entradas / salidas

In [ ]:
and_truth_table = [
        ((1, 1), [1]),
        ((1, -1), [-1]),
        ((-1, 1), [-1]),
        ((-1, -1), [-1])
    ]

tp2Aux.train_data_to_df(and_truth_table)

Pesos tras el entrenamiento. El 0 corresponde al bias, modelado como una entrada constante $x_0 = -1$

In [ ]:
tu_and=ThresholdUnit()
tu_and.train(and_truth_table)
tp2Aux.weights_to_df(tu_and)

La ecuación de la recta de decisión es $-w_0 + x_1 w_1 + x2_2 = 0$

In [ ]:
tp2Aux.plot_2d_tu(tu_and)

#### OR de dos entradas

Tabla de entradas / salidas

In [ ]:
or_truth_table = [
        ((1, 1), [1]),
        ((1, -1), [1]),
        ((-1, 1), [1]),
        ((-1, -1), [-1])
    ]

tp2Aux.train_data_to_df(or_truth_table)

Pesos tras el entrenamiento. El 0 corresponde al bias, modelado como una entrada constante $x_0 = -1$

In [ ]:
tu_or=ThresholdUnit()
tu_or.train(or_truth_table)
tp2Aux.weights_to_df(tu_or)

In [ ]:
tp2Aux.plot_2d_tu(tu_or)

#### AND de 4 entradas

Tabla de entradas / salidas

In [ ]:
and4_truth_table = [
    (( 1,  1,  1,  1), [ 1]),
    (( 1,  1,  1, -1), [-1]),
    (( 1,  1, -1,  1), [-1]),
    (( 1,  1, -1, -1), [-1]),
    (( 1, -1,  1,  1), [-1]),
    (( 1, -1,  1, -1), [-1]),
    (( 1, -1, -1,  1), [-1]),
    (( 1, -1, -1, -1), [-1]),
    ((-1,  1,  1,  1), [-1]),
    ((-1,  1,  1, -1), [-1]),
    ((-1,  1, -1,  1), [-1]),
    ((-1,  1, -1, -1), [-1]),
    ((-1, -1,  1,  1), [-1]),
    ((-1, -1,  1, -1), [-1]),
    ((-1, -1, -1,  1), [-1]),
    ((-1, -1, -1, -1), [-1]),
        
    ]

tp2Aux.train_data_to_df(and4_truth_table)

Pesos tras el entrenamiento. El 0 corresponde al bias, modelado como una entrada constante $x_0 = -1$

In [ ]:
tu_and4=ThresholdUnit()
tu_and4.train(and4_truth_table)
tp2Aux.weights_to_df(tu_and4)

#### OR de 4 entradas

In [ ]:
or4_truth_table = [
    (( 1,  1,  1,  1), [ 1]),
    (( 1,  1,  1, -1), [ 1]),
    (( 1,  1, -1,  1), [ 1]),
    (( 1,  1, -1, -1), [ 1]),
    (( 1, -1,  1,  1), [ 1]),
    (( 1, -1,  1, -1), [ 1]),
    (( 1, -1, -1,  1), [ 1]),
    (( 1, -1, -1, -1), [ 1]),
    ((-1,  1,  1,  1), [ 1]),
    ((-1,  1,  1, -1), [ 1]),
    ((-1,  1, -1,  1), [ 1]),
    ((-1,  1, -1, -1), [ 1]),
    ((-1, -1,  1,  1), [ 1]),
    ((-1, -1,  1, -1), [ 1]),
    ((-1, -1, -1,  1), [ 1]),
    ((-1, -1, -1, -1), [-1]),
    ]

tp2Aux.train_data_to_df(or4_truth_table)

In [ ]:
tu_or4=ThresholdUnit()
tu_or4.train(or4_truth_table)
tp2Aux.weights_to_df(tu_or4)

### 2. Determine numéricamente cómo varia la capacidad del perceptrón simple en función del número de patrones enseñados.

Se configuró una Threshold Unit (perceptron que utiliza la función signo) con $N = [1 .. 10]$ entradas.

Para cada configuración, y se entrena con K pares de entrada/salida  $(\mathbf{x}, y )$ aleatorios.

$\mathbf{x}$ es un vector de $N$ entradas aleatorias de distribución $U_{-1,1}$, $y$ una Bernoulli ${[-1,1]}$ con $P(\frac{1}{2})$

Mediante método de Montecarlo se estima la tasa de éxito de un entrenamiento de $K$ muestras aleatorias en un perceptrón de $N$ entradas, comenzando con $K=1$. El experimento se repite hasta obtener estimadores de la tasa de éxito con un error de $\pm 0.02$ en un intervalo de confianza del $95\%$

Para el caso de $N=1$ el método se validó contra el análisis analítico de la tasa de éxito. Se encontró que para 1000 de iteraciones y velocidad de convergencia $\eta=0.01$ seleccionados, el entrenamiento no encontraba solución a casos linealmente separables. Aún así se determinó que esos errores no modificaban considerablemente la salida del Montecarlo.

In [ ]:
dfCap=pd.read_csv("data/capacity.csv", usecols= lambda x: x in ["dim", "pmax", "cap"]).sort_values(["dim", "pmax"])

for dim,dim_cap in dfCap.groupby("dim"):
    plt.plot( dim_cap["pmax"]/dim_cap["dim"], dim_cap["cap"], label=int(dim))

plt.xlabel("pmax/N")
plt.ylabel("capacity")
plt.legend(title="N")
plt.title("Capacidad de una Threshold Unit")

Se observa que el ThresholdUnit tiene capacidad cercana a 1 para $\frac{\mathrm{pmax}}{N} \lt 1.5$

### 3. a) Implemente un perceptrón multicapa que aprenda la función lógica XOR de 2 y de 4 entradas (utilizando el algoritmo Backpropagation).

Se comienza reimplementando el perceptrón simple utilizando Non Linear Units para poder aplicar método del gradiente descendente sobre la función consto $C$

$C = \sum_k ( \zeta_k - g(\mathbf{w}^\mathrm{T} \cdot \mathbf{x_k}) )^2$

$\nabla C = \sum_k 2 ( \zeta_k - g(\mathbf{w}^\mathrm{T} \cdot \mathbf{x_k}))  g'(\mathbf{w}^\mathrm{T} \cdot \mathbf{x_k}) \mathbf{x_k}$

In [ ]:
nlu_or=NonLinearUnit()
nlu_or.train(or_truth_table)
tp2Aux.weights_to_df(nlu_or)